In [1]:
# 1. Import and install requirements
from logger import logger
from util.database import Database
from util.scraper import Scraper
import json
import itertools
import chromedriver_autoinstaller

database = Database("sqlite:///data/pdga_data.db")
scraper = Scraper()

chromedriver_autoinstaller.install()

/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[01/Nov/2024 15:01:59] INFO - Chromedriver is already installed.


'/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/chromedriver_autoinstaller/130/chromedriver'

In [ ]:
# 2. Build list of courses and write to JSON
courses = scraper.get_courses_from_dgscene()
course_names = {
    course: scraper.get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
# 3. Build list of events and write to JSON
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)

with open('data/course_events.json') as f:
    course_events: dict = json.load(f)

for i, course in enumerate(course_names):
    if course in course_events:
        logger.info(f'Skipping {course} (already scraped)...')
        continue

    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = scraper.get_all_sanctioned_events(course)
    logger.info(course_events[course])

    # periodically save ratings to file
    with open('data/course_events.json', 'w') as f:
        json.dump(course_events, f, indent=4)

logger.info("Done")

In [ ]:
# 4. Fetch ratings for each event and load into DB
try:
    logger.info('Fetching ratings...')
    
    with open('data/course_names.json') as f:
        course_names: dict = json.load(f)
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    
    for i, course in enumerate(course_events):
        events = course_events[course]
        rounds = []

        for j, event in enumerate(events):
            event_id = event['event_id']
            if database.event_exists(event_id):
                logger.info(f'Skipping {event_id} (already scraped)...')
                continue

            course_ratings = scraper.get_round_ratings_for_tournament(event_id)
            rounds.extend(course_ratings)
            logger.info(f'Event {j+1}/{len(events)} - Course {i+1}/{len(course_events)}')

        data = {
            'course_name': course,
            'readable_course_name': course_names[course],
            'events': course_events[course],
            'rounds': rounds
        }
        database.insert_course_data(data)
        

except BaseException as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')
except KeyboardInterrupt as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')

scraper.cleanup()
logger.info("Done")